# find tracerRNA

example id:
8774__OBKKDPJE_00146__BritoIL_2016__M2.46.ST__bin.47;length1073;#sequences1 

In [1]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/"




In [2]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy'],
      dtype='object')

In [3]:
tabellazza=pd.read_csv(datadir+"/3tabellazza/crisprcas_hits_table.csv", index_col=0)
tabellazza.columns

Index(['Unnamed: 0.1', 'Genome Name', 'Study', 'Sample Name', 'Assembler',
       'Genome Size', 'N50', '# Contigs', 'Completeness', 'Contamination',
       'Strain Heterogeneity', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_CRISPR', 'prokka_cas', 'uniref_cas'],
      dtype='object')

In [4]:
seqid ="LPEAHEHG_01071"
print("SEQ ID: ", seqid)
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
print("----------------------------------------------------------------------")


SEQ ID:  LPEAHEHG_01071
NODE_190_length_65757_cov_25.3876
ChengpingW_2017
ChengpingW_2017__AS43raw__bin.51
1074
[('.', '9802', '10102')]
----------------------------------------------------------------------


# LPEAHEHG_01071

In [5]:
seqid = "LPEAHEHG_01071"
spacers=[]
repeats=[]
repeat_start_pos=[]


cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")

######################################################################TODO THIS MUST BE INSIDE filename_discrepancies
# take into account filename discrepancies
if dataset.startswith("ZeeviD"):  #TODO guarda se funziona quando finisci di girare minced_runner
         s3_dataset="ZeeviD_2015"
         #genomename=filename_discrepancies.change_to_megahit(genomename)
         s3_genomename=genomename
         try:
             genomename=genomename.replace("ZeeviD_2015", "ZeeviD_2015_B")
             prokkafile_ofsample=open(annodir+"/justminced/ZeeviD_2015_B/"+genomename+".crisprcas.gff.minced")
             prokkafile_ofsample.close()
             dataset="ZeeviD_2015_B"
         except:
             genomename=genomename.replace("ZeeviD_2015_B", "ZeeviD_2015_A")
             dataset="ZeeviD_2015_A"
else:
    s3_dataset=dataset
    s3_genomename=genomename
    dataset=filename_discrepancies.s3(s3_dataset,r=True) # get working dataset name from s3 dataset name
    genomename=genomename.replace(s3_dataset, dataset)

if filename_discrepancies.dataset_has_megahit(dataset,genomename):
    genomename=filename_discrepancies.change_to_megahit(genomename)

samplename = genomename.split("__")[1]
########################################################################

# get CRISPR spacer and repeat sequence

mincedCRISPRfilename=datadir+"1crisprsearch/out/"+s3_dataset+"/"+s3_genomename+".fa.minced.out"
f=open(mincedCRISPRfilename, "r")
for line in f.readlines():
    print(line)
    if line[0]== "1" or line[0]== "2" or line[0]== "3" or line[0]== "4" or line[0]== "5" or \
    line[0]== "6" or line[0]== "7" or line[0]== "8" or line[0]== "9" or line[0]== "0":

        repeats.append(line.split("\t")[2])

        repeat_start_pos.append(line.split("\t")[0])

        if not line.split("\t")[3]=="\n":
            spacers.append(line.split("\t")[3])
f.close()

# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai genomename della cas e/o seqid della cas (anzi famo solo seqid, che magari ci sn doppioni in genomi) e ti dà tt il locus

# print start and stop of CRISPR
unostranosplit=cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0].split("\', \'")
crispr_start, crispr_stop = int(unostranosplit[1]), int(unostranosplit[2].strip("\')]\""))

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.

prokka_anno=tabellazza[tabellazza["Genome Name"]==s3_genomename]["prokka_cas"].iloc[0]
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così

prokkaslist=prokka_anno.split(">")
prokkaslist.pop(0)
# get contigname of current effectorcas   
cas_position={}
for casanno in prokkaslist:
    unostranosplit=casanno.split("Name=")
    casnumber=unostranosplit[1][3] #because we know that we are loooking at 'good' loci, this is just going to be 9, 1 or 2
    unaltrostranosplit=casanno.split("\t")
    cas_start=unaltrostranosplit[3]
    cas_stop=unaltrostranosplit[4]
    cas_position["Cas"+casnumber]=[int(cas_start),int(cas_stop)]

genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
print("Retrieving locus")


# access contig:

for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, "/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb", "fasta")
        print("CRISPR si trova", crispr_start, crispr_stop)
        print("Cas9 si trova", cas_position["Cas9"])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        cas9start=[int(x) for x in cas_position["Cas9"]][0]
        cas9stop=[int(x) for x in cas_position["Cas9"]][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()
 
  
        print(len(cas9_nnseq)/3,len(cas9_aa))

        # check strand for all Cas9s (scriveresti ben meno codice a metterlo in una funzioncina, ma ok)
        if my_translated_cas9==cas9_aa:
            print("Cas9 locus on plus strand")
            plus=True
        else:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas9 locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)

        #extact a locus, 200 bp a monte e a valle dei cosi che ho:
        starts, stops=[startstop[0]for startstop in cas_position.values()], [startstop[1]for startstop in cas_position.values() ]
        minimo=min([crispr_start]+starts)
        print("start: ", minimo)
        massimo=max([crispr_stop]+stops)
        print("end: ", massimo)

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        blast_folder="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"
        os.chdir(blast_folder)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")
        for n, repeat in enumerate([Seq(sequence) for sequence in np.unique(repeats)]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db
        print("Running blastn of query file agianst all contigs")

        #3. make db file
        dbfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb"
        os.system("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")



        blastoutfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"\
        +str(SGB)+"__"+seqid+"__"+genomename+"__"+feature+".trcr.blastout"
        blastn_command = "blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11"
        print(blastn_command)
        os.system(blastn_command)
       # os.system("rm /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracerRNA/temp*")



-----------------------------------------------------
Cas9 id:	LPEAHEHG_01071
SGB:	15299
Genome Name:	 ChengpingW_2017__AS43raw__bin.51
Contig:	NODE_190_length_65757_cov_25.3876 
-----------------------------------------------------

Sequence 'NODE_190_length_65757_cov_25.3876' (65757 bp)



CRISPR 1   Range: 9802 - 10102

POSITION	REPEAT				SPACER

--------	------------------------------------	------------------------------

9802		GGGATGATTTACTAATAGCTCTTGGTATGGTATAAT	CTTCCGTGCCATTTCCTCTATGACAGAATC	[ 36, 30 ]

9868		GTCATAGTTCCCTAACAGCTCTTGGTATGGTATAAT	CTGCCCGGCGACGTGCGTGTGAGCGATACG	[ 36, 30 ]

9934		GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT	CCTGCCGCCCAATGCCCAGGACAACGAACT	[ 36, 30 ]

10000		GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT	AACGACTATTCAACGCACGGTAATAGCGCCT	[ 36, 31 ]

10067		GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT	

--------	------------------------------------	------------------------------

Repeats: 5	Average Length: 36		Average Length: 30



Time to find repeats: 106 ms





Retrieving l

In [6]:
#4. parse blast output: only print output if it is NOT one of the old repeats.
print("\n-----------------------------------------------------\nParsing BLAST output, looking for hit which are NOT the same repeats\ni.e. not starting with:", repeat_start_pos,":\n-----------------------------------------------------\n")
f=open(blastoutfile,"r")
for line in f.readlines():
    start_pos=line.split("\t")[9]
    if start_pos not in repeat_start_pos:
        if str(int(start_pos)+1) not in repeat_start_pos:
            if str(int(start_pos)-1) not in repeat_start_pos:
                print(line) 


-----------------------------------------------------
Parsing BLAST output, looking for hit which are NOT the same repeats
i.e. not starting with: ['9802', '9868', '9934', '10000', '10067'] :
-----------------------------------------------------

rpt1|NODE_190_length_65757_cov_25.3876|ChengpingW_2017__AS43raw__bin.51|LPEAHEHG_01071	NODE_190_length_65757_cov_25.3876	100.00	36	25	0	0	CTAATAGCTCTTGGTATGGTATAAT	CTAATAGCTCTTGGTATGGTATAAT	9945	9969	9e-09	plus

rpt1|NODE_190_length_65757_cov_25.3876|ChengpingW_2017__AS43raw__bin.51|LPEAHEHG_01071	NODE_190_length_65757_cov_25.3876	100.00	36	25	0	0	CTAATAGCTCTTGGTATGGTATAAT	CTAATAGCTCTTGGTATGGTATAAT	10011	10035	9e-09	plus

rpt1|NODE_190_length_65757_cov_25.3876|ChengpingW_2017__AS43raw__bin.51|LPEAHEHG_01071	NODE_190_length_65757_cov_25.3876	100.00	36	25	0	0	CTAATAGCTCTTGGTATGGTATAAT	CTAATAGCTCTTGGTATGGTATAAT	10078	10102	9e-09	plus

rpt1|NODE_190_length_65757_cov_25.3876|ChengpingW_2017__AS43raw__bin.51|LPEAHEHG_01071	NODE_190_length_65757_cov

## extract sequences

In [7]:
matched_rep_start=14729
matched_rep_stop=14751

In [8]:
# this is the sequence matching the repeat:
matched_rep=Seq("TAATAGTTCTTGGTATGGTATAA")

#rpts which match:
rpt2=Seq("GTCATAGTTCCCTAACAGCTCTTGGTATGGTATAAT")
rpt3=Seq("GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT")

# since repeat is on the + strand, and this is all on the + strand, tracrRNA should be its reverse complement:
antirepeat=matched_rep.reverse_complement()
#this should go frombo to bo  = x nt  (x+1)
print(antirepeat)
len(antirepeat)

TTATACCATACCAAGAACTATTA


23

In [9]:
rpt1=Seq("GGGATGATTTACTAATAGCTCTTGGTATGGTATAAT")

In [10]:
maturerpt3=rpt3[12:]  #sono gli ultimi 24 nt (see paper Processing-Independent CRISPR RNAs limit natural transformation in N. meningitidis)
# update: non è veramente il mature repeat. il mature repat sarebbero i PRIMI 24...)
realmaturerpt3=rpt3[:24]

In [11]:
realmaturerpt3, len(realmaturerpt3)

(Seq('GTCATAGTTCCCTAATAGCTCTTG'), 24)

In [12]:
realmaturerpt3.reverse_complement()

Seq('CAAGAGCTATTAGGGAACTATGAC')

In [13]:
# print terminators

f=open(blast_folder+str(SGB)+"__"+seqid+"__"+genomename+".terminators")
for line in f.readlines():
    if line.strip().startswith("14"):
        print(line)
f.close()

14653      Both - CGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCaTCTTTTTTGTCA -14.40

14719     Erpin - TATACTATAAACAGTTCTTATACCATACCAAGAACTATTATGGTTGCTAT NA

14722      Both + GCAACCATAATAGTTCTTGGTATGGTATAAGAACTgTTTATAGTATACA -7.40



In [14]:
# this is the rho-independent trnascription terminator, going from 14719 or 14722 to bo (+1 perché conti il primo tipo), 

terminator_end=14652 # (sarebe 1463-1, for indexing reasons) sarebbe la fine xke prendiamo il rev comp remembra. che ha senso xke l'antirepeat è a 14729, che è dopo
 #oppure, ce n'è uno con TTTTTTT a 14653... 
terminator=Seq("CGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCaTCTTTTTT".upper())
print(len(terminator))
terminator_start=terminator_end+len(terminator) #occhio che qui start e stop si riferisce al rev.compl, mentre nel antirepeat si riferisce alla direzione base

50


In [3]:
a="123456789"
a[:8-4]

'1234'

In [15]:
# Open genome and extract the whole sequence
# find this inside the genome, and link it to the thing found by ARNold:
for record in SeqIO.parse("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/selected_loci/"+seqid+"/"+genomename+".fa", "fasta"):
    if record.id==contigname:
        contigseq=record.seq

#check antirepeat
print("test repeat: ", antirepeat, len(antirepeat), contigseq[matched_rep_start-1:matched_rep_stop].reverse_complement()==antirepeat) # -1 why? Indexing diverso evidentemente col blastout, vbb

# check terminator
print("test terminator: " ,terminator, len(terminator), contigseq[terminator_end:terminator_start].reverse_complement()==terminator)


pre_tracrRNA=contigseq[terminator_end:matched_rep_stop].reverse_complement()
print("pre tracrRNA:", pre_tracrRNA, len(pre_tracrRNA))

# add mario
mario=contigseq[terminator_start:matched_rep_start-1].reverse_complement()
print("mario sequence:",  mario, len(mario))  # ho fatto i check, il -1 sopra è giusto

test repeat:  TTATACCATACCAAGAACTATTA 23 True
test terminator:  CGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCATCTTTTTTGTCA 50 True
pre tracrRNA: TTATACCATACCAAGAACTATTATGGTTGCTATGATAAGGTCATAGGACCGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCATCTTTTTTGTCA 99
mario sequence: TGGTTGCTATGATAAGGTCATAGGAC 26


In [16]:
#mario sequence, conta fino a che antirepeatè+mario=36 nt, e prendi quelli dopo, guarda caso cominciano per AA. è un bell'indicatore che sia una mario sequence.

ora ha una lunghezza simile, e finisce in modo simile a quello di _S. pyogenes_  TTTTTTTaltre

In [18]:
alignments = pairwise2.align.localxx(rpt3, (antirepeat+mario[:12]).reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt3))


GTCATAGTTC--CC-TAATAGCT-CTTGGTATGGTATAAT
 ||||||  |  || |||||| | |||||||||||||||
-TCATAG--CAACCATAATAG-TTCTTGGTATGGTATAA-
  Score=31

rpt length: 36


In [17]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch

alignments = pairwise2.align.localxx(maturerpt3, antirepeat.reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt3))


TAATAGCT-CTTGGTATGGTATAAT
|||||| | |||||||||||||||
TAATAG-TTCTTGGTATGGTATAA-
  Score=22

rpt length: 36


In [124]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch

alignments = pairwise2.align.localxx(realmaturerpt3.reverse_complement(), antirepeat.reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt3))


CAAG-AG--CTAT--TAG-GGAACTATGAC
 ||  ||  || |  ||  ||   ||| |.
TAA-TAGTTCT-TGGTA-TGG---TAT-AA
  Score=15

rpt length: 36


In [115]:
maturerpt3.reverse_complement()

Seq('ATTATACCATACCAAGAGCTATTA')

In [103]:
# align antirepat + mario with repeat
alignments = pairwise2.align.globalxx(maturerpt3, (antirepeat+mario).reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))

-T---A--A--T-A-----GC-----T-------CTTGGTATGGTATAAT
 |   |  |  | |     ||     |       ||||||||||||||| 
GTCCTATGACCTTATCATAGCAACCATAATAGTTCTTGGTATGGTATAA-
  Score=23



### compare with known spCas9 sgRNA

In [7]:
#allignment with official sgRNA
spCas9_sgRNA=Seq("GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGCTTTTTTT")

#which is made by:
spCas9_tracrRNA=spCas9_sgRNA[16:]
spCas9_tetraloop=spCas9_sgRNA[12:16]
spCas9_repeat=spCas9_sgRNA[:12]

In [17]:
alignments = pairwise2.align.localxx(spCas9_tracrRNA, pre_tracrRNA)
print(pairwise2.format_alignment(*alignments[0]))


T-AGC-A--AGTTA--AA-A--TA--A-GGCTAGTCCG-T-T-ATCAAC--T--TGA--A----AAAG-T--G--GCACC-G----AGT--C-GGTGC-T--T-TTTT---T--
| |   |  |  ||  || |  ||  | || |  |  | | | || ||   |  | |  |    |||| |  |  || || |    ||   | || || |  | ||||   |
TTA--TACCA--TACCAAGAACTATTATGG-T--T--GCTATGAT-AA-GGTCAT-AGGACCGTAAAGCTCTGACGC-CCTGTCCTAG-GACAGG-GCGTCATCTTTTTTGTCA
  Score=52



### compar ewoith known NMCas9 sgRNA

In [65]:
#allignment with official  stuff (efficient gene editing balbla cs9 neisseria meningitidis, Hou et al. 2013)
nmCas9_repeat=Seq("GTTGTAGCTCCCTTTCTCATTTCG")
nmCas9_unprocessed_tracrRNA=Seq("AUAUUGUCGCACUGCGAAAUGAGAACCGUUGCUACAAUAAGGCCGUCUGAAAAGAUGUGCCGCAACGCUCUGCCCCUUAAAGCUUCUGCUUUAAGGGGCAUCGUUUA")
nmCas9_proc_tracrRNA=Seq("AAAUGAGAACCGUUGCUACAAUAAGGCCGUCUGAAAAGAUGUGCCGCAACGCUCUGCCCCUUAAAGCUUCUGCUUUAAGGGGCAUCGUUUA")
print("repeat", len(nmCas9_repeat))
print("unprocessed: ", len(nmCas9_unprocessed_tracrRNA))
print("processed: ", len(nmCas9_proc_tracrRNA))
alignments = pairwise2.align.localxx(nmCas9_unprocessed_tracrRNA, nmCas9_proc_tracrRNA)
print(pairwise2.format_alignment(*alignments[0]))


repeat 24
unprocessed:  107
processed:  91
AUAUUGUCGCACUGCGAAAUGAGAACCGUUGCUACAAUAAGGCCGUCUGAAAAGAUGUGCCGCAACGCUCUGCCCCUUAAAGCUUCUGCUUUAAGGGGCAUCGUUUA
                |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
----------------AAAUGAGAACCGUUGCUACAAUAAGGCCGUCUGAAAAGAUGUGCCGCAACGCUCUGCCCCUUAAAGCUUCUGCUUUAAGGGGCAUCGUUUA
  Score=91



In [52]:
len(mario+terminator)

76

In [64]:
alignments = pairwise2.align.localxx(randomDNA(76), randomDNA(76))
print(pairwise2.format_alignment(*alignments[0]))

-TG-C-T-TTGTAGTCTA-ATGG-GAC-GCGT-CGGG--CCCTA-ACTG-C-GCCACAGCCT-TTACC--T-A---TC-CCATGTGCGACTGATA---G-A-
 || | | || ||  ||  | || | | ||   |     |||   |||  | ||| ||   | ||  |  | |   || |||     ||||| ||   | |
GTGTCGTCTT-TA--CT-GA-GGTG-CAGC--AC---TTCCC--GACT-TCCGCC-CA---TGTT--CGGTTAAAATCGCCA-----GACTG-TAACTGGAT
  Score=50



In [60]:
# geanerate a random N nucleotides sequence
def randomDNA(N):
    s=""
    d={1:"A",2:"C",3:"G",4:"T"}
    for n in range(N):
        s+=str(d[np.random.randint(1,5)])
    return s


In [88]:
randomDNA(76)

'CAGTGTTAATCCCAAATTCAATGCTATATAGAATACTCCTAGCGCAAGGACCCGGATCCAGGGCTCTGATGCGCAA'

In [109]:
# just un check pratico per vedere che effettivamente la probabilità di avere una precisa sequenza di lunghezza N è (1/4)^N
mean=[]
lungh_seq=2 
# in teoria, deve uscire 1
for tentativo in range(10):
    n=0
    for i in range(4**lungh_seq):
        if randomDNA(lungh_seq)=="T"*lungh_seq:
            n+=1
    print("number "+"T"*lungh_seq+" sequnces:", n)
    mean.append(n)
print("mean n of TTTT sequences for 4^4 throws:", np.mean(mean))

number TT sequnces: 1
number TT sequnces: 1
number TT sequnces: 0
number TT sequnces: 2
number TT sequnces: 1
number TT sequnces: 1
number TT sequnces: 2
number TT sequnces: 1
number TT sequnces: 1
number TT sequnces: 1
mean n of TTTT sequences for 4^4 throws: 1.1


In [104]:
alignments = pairwise2.align.localxx(nmCas9_proc_tracrRNA.transcribe(), pre_tracrRNA)
print(pairwise2.format_alignment(*alignments[0]))


AAAUG-AG--A-ACCGUUGCUACA-AUA--A----GGCC--GUCUG-A--A-AAG---AU--GUG-CCGC-AAC-GCU-CU-G-C-CCCUUAAA-G-CUUCU--GCUUU-A-AGGG-G-CAU-C------GUUU--A
  |   |   | | |    | | | | |  |    ||    | |   |  | |||   |   | | |||  ||  ||  |  | | |||      | |  |   |     | |||| | ||  |      |     |
TTA--TA-CCATA-C----C-A-AGA-ACTATTATGG--TTG-C--TATGATAAGGTCA-TAG-GACCG-TAA-AGC-TC-TGACGCCC-----TGTC--C-TAG----GACAGGGCGTCA-TCTTTTTTG---TCA
  Score=51



## Final tracrRNA

In [125]:
# I would probably say that the tracr is just, simply la parte di pre_tracrRNA la cui antirepeat si lega alla repeat, 
# che, in questo caso, è tutta.
tracrRNA = pre_tracrRNA

print("\n----------------------------------------------------------------------")
print("         tracrRNA sequence for", feature,"protein id", seqid)
print("----------------------------------------------------------------------\n")
print("- contig name:", cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print("- Dataset: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print("- Bin: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print("- SGB: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print("----------------------------------------------------------------------\n")
print("- Length: ", len(tracrRNA),"\n- position: ", terminator_end,matched_rep_stop)
print(tracrRNA)


----------------------------------------------------------------------
         tracrRNA sequence for Cas9 protein id LPEAHEHG_01071
----------------------------------------------------------------------

- contig name: NODE_190_length_65757_cov_25.3876
- Dataset:  ChengpingW_2017
- Bin:  ChengpingW_2017__AS43raw__bin.51
- SGB:  15299
----------------------------------------------------------------------

- Length:  99 
- position:  14652 14751
TTATACCATACCAAGAACTATTATGGTTGCTATGATAAGGTCATAGGACCGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCATCTTTTTTGTCA


se pensi di Do the same, ma sullo strand opposto, xke non sappiamol'orientamento di CRISPR array, ma sappiam oche i cas9 è sul - , quindi se anche crispr fosse su-, tracr dovrebbe essere su +.
it's on the same strand as  Cas9... quindi se  facessi le stesse analisi di cui sopra, ma senza prendere nessun reverse comple,ent (poiché Cas9 and friends sono sullo strand - (anche se non è dettoxke CRISPR su che strand è??) )
orientation of CRISPR is not so easy to decide, e nel nostro caso direi che non possiamo saperla. quindi bella.

ma NO! perché il terminator viene prima dell'antirepeat, quindi la sequenza ddev'essere reverse complement per forza!

In [22]:
#altra cosa a supporto del mio oso: 'g23' guide sequence format per la cosa. belllllla (citazione da IL paper, chje cita altri 3 papers (ctrl+f guide))

In [23]:
# se riuscissi atrovare tutti i tracrRNA, che siano abbastranza per allenare una rete? O:

## print output

In [131]:
outputdir=datadir+"8outputv1/"+seqid+"/"
os.system("mkdir "+outputdir)
f=open(outputdir+seqid+"_tracr_nt.txt","w")
f.write(">"+sequid+"SGB="+SGB+" len="+length+"\n"+tracrRNA)
f.close()

In [132]:
SGB

15299